In [15]:
!nvidia-smi

Tue Nov  3 05:28:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
pip install transformers

     |████████████████████████████████| 1.3MB 3.2MB/s 
     |████████████████████████████████| 1.1MB 10.7MB/s 
     |████████████████████████████████| 890kB 22.4MB/s 
     |████████████████████████████████| 2.9MB 28.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b528ea3f8b38f6a1a484a8bfc5f2803f9c9a29ddd405494009ec4cadd2949ec6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
import warnings
import time
warnings.filterwarnings("ignore")

In [3]:
def dataset(filename):
    with open(filename,'r') as fp:
        lines = [line.strip() for line in fp]
    return lines

In [4]:
def wordData(data):
    wordLines = data
    words = []
    probabilities = []
    wordList = []
    pos = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]
            prob = lineSplit[4]
            temp = lineSplit[5]
            words.append(word)
            probabilities.append(prob)
            pos.append(temp)
        elif not (len(empty) and []):
            wordList.append((words, pos, probabilities))
            words = []
            probabilities = []
            pos = []
    return wordList

In [5]:
def wordtest(data):
    wordLines = data
    words = []
    testWord = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]            
            words.append(word)
        elif not len(empty):
            testWord.append(words)
            words = []       
    return testWord

In [6]:
def preTokenizing(data):
    t = []
    po = []
    prob = []
    for i,j,k in data:
        text = []
        pos = []
        probs = []
        for l in i:
            text.append(l)
        for m in j:
            pos.append(m)
        for n in k:
            probs.append(float(n))
        t.append(text)
        po.append(pos)
        prob.append(probs)
    return t,po, prob

In [7]:
TRAINING_FILE = "train.txt"
DEV_FILE = "dev.txt"
TEST_FILE = "test_data.txt"

tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-large-en")


In [8]:
trainText = wordData(dataset(TRAINING_FILE))
testEval = wordtest(dataset(TEST_FILE))
devText = wordData(dataset(DEV_FILE))


trainWords,trainTags, trainLabels = preTokenizing(trainText)
devWords, devTags, devLabels = preTokenizing(devText)

tokenized_text = []
for i in trainWords:
    sent = ""
    for j in i:
        if sent == "":
            sent += j
        else:
            sent = sent + " " + j
    tokens = tokenizer.tokenize(sent)
    tokenized_text.append(tokens)

    

# Encoding each word of the sentence and appending to a list
en=[]
for i in tokenized_text:
  en1 = []
  for j in i:
    token_ids = en1.append(tokenizer.encode_plus(j, add_special_tokens=False, return_attention_mask=False, return_tensors='pt'))
  en.append(en1)
  en1 = []
  
# Getting the input id's only for the list and passing them to the model 
words = []  
for d in en:
    words2 = []
    for i in d:
        words2.append(i['input_ids'])
    words.append(words2)
    words2 = []     

In [9]:
class BertBinaryClassifier(nn.Module):
    def __init__(self):
        super(BertBinaryClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('nghuyong/ernie-2.0-large-en')
        self.linear = nn.Linear(1024, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens):
        _, pooled_output = self.bert(tokens)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        proba = proba.data.tolist()
        return proba

In [ ]:
bert_clf = BertBinaryClassifier()
optimizer = optim.Adamax(bert_clf.parameters(), lr=0.0001)
bert_clf.train()
token_answers = []
for epoch_num in range(10):
    start_time = time.time()
    print("Running epoch_num ---->{}.....".format(epoch_num))
    count = 0
    ep_ls = 0.0
    for batch_data, batch_probs in zip(words, trainLabels):
        bert_clf.zero_grad()
        answers_temp = []
        for i in batch_data:
            probas = bert_clf(i)
            for item in probas:
                for i in item:
                    answers_temp.append(i)            
        token_answers.append(answers_temp)
        try:
          loss_func = nn.MSELoss(reduction = 'sum')
          batch_loss = loss_func(torch.tensor(answers_temp), torch.tensor(batch_probs))
          ep_ls = ep_ls + batch_loss.item()
          optimizer.step()
          count = count + 1
        except:
          continue
    print("loss ---->{}\n".format((ep_ls/float(count))))
    print("Total runtime ----> %s seconds ---" % (time.time() - start_time))
      
      


Running epoch_num ---->0.....
loss ---->1.6340735800930482

Total runtime ----> 5078.634051084518 seconds ---
Running epoch_num ---->1.....
loss ---->1.6308940829996856

Total runtime ----> 5059.952414512634 seconds ---
Running epoch_num ---->2.....
loss ---->1.6335824734231623

Total runtime ----> 4960.302845001221 seconds ---
Running epoch_num ---->3.....
loss ---->1.6282946733642751

Total runtime ----> 5052.599014997482 seconds ---
Running epoch_num ---->4.....
loss ---->1.6275021723492056

Total runtime ----> 4962.310858011246 seconds ---
Running epoch_num ---->5.....
